#Put your Google Colab link here:
*your link here*

## Important notice: any use of generative AI for completing the assignment is strictly prohibited.

# DOCTOR: A Multi-Disease Detection Continual Learning Framework Based on Wearable Medical Sensors

In this exercise, we will use PyTorch to recreate some of the experiments done in the [DOCTOR](https://dl.acm.org/doi/full/10.1145/3679050) paper. We will perform domain-, class-, and task-incremental learning for a multilayer perceptron (MLP) model detecting diabetes and mental health disorders using replay-based continual learning methods.

*   Navigate to the tabs above. Click `Runtime` -> `Change runtime type` -> select `GPU` as the hardware accelerator to enable GPU, which will allow you to train faster.

# Part 1: Prepare data (0.5 pt)

##Import useful libraries:

In [ ]:
import os
import math
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random

from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
from torch.nn import Module, ReLU, Linear, LogSoftmax, CrossEntropyLoss

from sys import getsizeof
from itertools import cycle
from scipy.stats import kstest
from sklearn.utils import shuffle
from sklearn.neighbors import KernelDensity
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture as GMM
from sklearn.metrics import confusion_matrix, classification_report

##Get access to a GPU:
To gain access to the GPUs on Colab, navigate to the `Runtime` tab above and select `Change runtime type`.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Prepare the experimental datasets:

### Experimental datasets:

For this exercise, we use the DiabDeep dataset collected for the [DiabDeep](https://ieeexplore.ieee.org/abstract/document/8935429) project. We will use the DiabDeep dataset for the domain- and class-incremental learning experiments.

*  **The DiabDeep Dataset:**
The DiabDeep dataset contains physiological signals and environmental information collected from 25 non-diabetic individuals, 14 Type-I diabetic patients, and 13 Type-II diabetic patients with a smartwatch and a smartphone.

The datasets used here are their streamlined versions. The DiabDeep dataset contains 20,957 data instances with 4,485 features. Refer to Table 2 in [DOCTOR](https://dl.acm.org/doi/full/10.1145/3679050) for the data features included in these datasets.

### Data preprocessing:

It is crucial and a good practice to [preprocess](https://neptune.ai/blog/data-preprocessing-guide) your dataset before you jump into model training. Data preprocessing includes handling missing values, data nomalization, feature selection, dimensionality reduction, etc. This part has been done for you. See Section 5.2 in [DOCTOR](https://dl.acm.org/doi/full/10.1145/3679050) for the details about the dataset preprocessing that had been done.

### Prepare data for experiments:

* **Domain-incremental learning:**
For the domain-incremental learning experiment, we split the DiabDeep dataset into two missions (domains) in a stratified fashion. Mission-1 contains data from 80% of the patient files (20 healthy, 11 Type-I, and 10 Type-II), while Mission-2 contains data from the other 20% of the patient files (5 healthy, 3 Type-I, and 3 Type-II). Then, we take the first 70%, the next 10%, and the last 20% of each patient's sequential time series data to construct the training, validation, and test sets with no time overlap.

* **Class-incremental learning:**
For the class-incremental learning experiment, we split the dataset into two missions. Mission-1 contains data from healthy individuals and Type-I diabetic patients, while Mission-2 includes solely the Type-II diabetic patients. Similarly, we take the first 70%, the next 10%, and the last 20% of each patient's sequential time series data to construct the training, validation, and test sets with no time overlap.

##Create a custom dataset class: (0.5 pt)
To prepare our datasets, we create a `CustomDataset` class that inherits PyTorch's [Dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) class.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        # Transform x, y to torch tensors
        """TO DO"""
        self.x =
        self.y =

    def __len__(self):
        # Return the len of the dataset
        """TO DO"""
        return

    def __getitem__(self, index):
        return self.x[index, :], self.y[index]

# Part 2: Domain-Incremental Learning Experiment (14 pts)

##Load the experimental data and create the datasets and dataloaders: (1 pt)
The dataset has been processed into the training, validation, and test sets for the two domains for you. You just need to load the corresponding experimental data.

First, we use Numpy to load the experimental data. Then, we create the training, validation, and test sets using the `CustomDataset` class. Finally, we instantiate dataloaders with PyTorch's [DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip the dataset
!unzip "/content/drive/Shared drives/ECE477 datasets/Assignment6/diabdeep_DIL_data.zip" -d diabdeep_DIL_data

# Check out the experimental data
print('\n', os.listdir('diabdeep_DIL_data/'))

# Load the data features x from Mission-1 (Domain-1) as type "float32"
"""TO DO"""
x_train_1 =
x_valid_1 =
x_test_1 =

# Load the data features x from Mission-2 (Domain-2) as type "float32"
"""TO DO"""
x_train_2 =
x_valid_2 =
x_test_2 =

# Load the labels y from Mission-1 (Domain)-1 as type "int64"
"""TO DO"""
y_train_1 =
y_valid_1 =
y_test_1 =

# Load the labels y from Mission-2 (Domain)-2 as type "int64"
"""TO DO"""
y_train_2 =
y_valid_2 =
y_test_2 =

# Inspect the labels for both missions
print(f"Mission-1 labels: {np.unique(y_test_1)}")
print(f"Mission-2 labels: {np.unique(y_test_2)}")

# check dimensions
print(f"x_train_1 shape: {x_train_1.shape}")

# Instantiate the datasets with CustomDataset
"""TO DO"""
train_1 =
valid_1 =
test_1 =
train_2 =
valid_2 =
test_2 =

# Set the batch size to 128
batch_size = 128

# Instantiate the dataloaders
# Note that we shuffle all of our datasets except for train_loader_1
# since we need to record each training data instance's training loss
# over all epochs for data selection (details later)
# Note that we halve the batch size for train_loader_2
# since we will train the model with a balanced amount of data from both domains
# in each batch for continual learning
"""TO DO"""
train_loader_1 =
valid_loader_1 =
test_loader_1 =
train_loader_2 =
valid_loader_2 =
test_loader_2 =

##Create an MLP model: (1.5 pts)

Here, we will build a four-layer MLP model. See [here](https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html) for an example of building a deep neural network with PyTorch.



*   Use [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) to define a fully-connected feed-forward layer
*   Use [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) for the activation function



In [ ]:
class MLP(Module):
    def __init__(self, in_num, out_num, hidden_num1, hidden_num2, hidden_num3):
        super(MLP, self).__init__()
        # Add the first hidden layer and ReLU activation
        """TO DO"""
        self.hidden1 =
        self.relu1 =

        # Add the second hidden layer and ReLU activation
        """TO DO"""
        self.hidden2 =
        self.relu2 =

        # Add the third hidden layer and ReLU activation
        """TO DO"""
        self.hidden3 =
        self.relu3 =

        # Add the output layer
        """TO DO"""
        self.out =

    def forward(self, x):
        # Pass x forward the first hidden layer and ReLU activation
        """TO DO"""

        # Pass x forward the second hidden layer and ReLU activation
        """TO DO"""

        # Pass x forward the third hidden layer and ReLU activation
        """TO DO"""

        # Pass x forward the output layer
        """TO DO"""

        return x

##Define the accuracy evaluation function: (1 pt)

Here, we define the helper function `acc_eval()` that evaluates the model's accuracy.

In [ ]:
def acc_eval(mlp, loader, set_len):
    '''
        Evaluate accuracy.

        Args:
            mlp:        The MLP model
            loader:     DataLoader
            set_len:    Number of data in the dataset
        Returns:
            accuracy:   Accuracy
    '''
    correct = 0
    # Set mlp model to evaluation state
    """TO DO"""


    with torch.no_grad():
        for (x, y) in loader:
            # Cast the features x to `device`
            """TO DO"""

            # Cast the labels y to `device`
            """TO DO"""

            # Use the MLP model to predict on x and get the predicted labels y_hat (which has the max probability)
            """TO DO"""
            y_hat =
            correct += (y_hat == y).int().sum()
    accuracy = correct / set_len
    return accuracy

##Initialize parameters:
We initialize some of the parameters we will use in this exercise here.

In [ ]:
random_state = 0      # Set random state to 0
learning_rate = 5e-3  # Initial learning rate for the SGD optimizer
momentum = 9e-1       # Momentum for the SGD optimizer
epoch = 30           # Number of epochs

##Define the training pipeline: (2 pts)

We define the training function `train_model()` that we'll use to train our MLP model here.

*   Use the [stochastic gradient descent optimizer](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html) with momentum
*   Use the [cross entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) as our loss function

In [ ]:
def train_model(mlp, epoch, train_loader, valid_loader, train_set_len, valid_set_len,
                file_name, verbose=True, save_loss=False, labels=None, pre_loader=None):
    '''
        Train the neural network.

        Args:
            mlp:            The MLP model
            epoch:          Number of epochs
            train_loader:   DataLoader for training data
            valid_loader:   DataLoader for validation data
            train_set_len:  Number of data in the training dataset
            valid_set_len:  Number of data in the validation dataset
            file_name:      File name to store the state dict
            verbose:        See the training progress or not
            save_loss:      Save the loss matrix or not
            labels:         Labels of the training data if save_loss == True
            pre_loader:     DataLoader for synthetic data representing previous task
        Returns:
            tra_acc:        Training accuracy throughout all epochs
            val_acc:        Validation accuracy throughout all epochs
            _loss_matrix:   Loss matrix storing the average training loss
    '''
    # Initialize the SGD optimizer
    """TO DO"""
    opt =
    # Initialize the cross entropy loss
    """TO DO"""
    lossF =
    # Initialize two lists to record training accuracy and validation accuracy, respectively
    tra_acc, val_acc = [], []

    # Perform generative replay continual learning with data from pre_loader
    if pre_loader:
        for e in range(1, epoch + 1):
            # Set the mlp model to training state
            """TO DO"""

            # Initialize the running loss of the current epoch to 0.0
            running_loss = 0.0
            # Get training data from both the train_loader and pre_loader
            for i, (data1, data2) in enumerate(zip(train_loader, cycle(pre_loader)), 1):
                # Zero out the gradients
                """TO DO"""

                x1, y1 = data1
                x2, y2 = data2
                # Concatenate features (x1, x2) from both DataLoaders along the correct dimension
                """TO DO"""
                x =
                # Concatenate labels (y1, y2) from both DataLoaders along the correct dimension
                """TO DO"""
                y =
                # Cast the features to `device`
                """TO DO"""

                # Cast the labels to `device`
                """TO DO"""


                # Calculate the loss
                """TO DO"""
                loss =
                # Back propagate the gradient
                """TO DO"""


                # Update the model
                """TO DO"""

                # Accumulate the running loss
                running_loss += loss.item()

                # Save training loss for data selection later
                if save_loss:
                    _lossF = CrossEntropyLoss(reduction='none')
                    _loss_tr = _lossF(mlp(x1.to(device)), y1.to(device)).cpu().detach().numpy().reshape((-1, 1))
                    _loss_pr = _lossF(mlp(x2.to(device)), y2.to(device)).cpu().detach().numpy().reshape((-1, 1))
                    if i == 1:
                        tmp_loss_tr_matrix = np.copy(_loss_tr)
                        tmp_loss_pr_matrix = np.copy(_loss_pr)
                    else:
                        tmp_loss_tr_matrix = np.concatenate((tmp_loss_tr_matrix, _loss_tr), axis=0)
                        tmp_loss_pr_matrix = np.concatenate((tmp_loss_pr_matrix, _loss_pr), axis=0)

            # Save training loss for data selection later
            if save_loss:
                if e == 1:
                    _loss_tr_matrix = np.copy(tmp_loss_tr_matrix)
                    _loss_pr_matrix = np.copy(tmp_loss_pr_matrix)
                else:
                    _loss_tr_matrix += tmp_loss_tr_matrix     # Accumulate loss
                    _loss_pr_matrix += tmp_loss_pr_matrix     # Accumulate loss
                if e == epoch:
                    _loss_tr_matrix = _loss_tr_matrix / e     # Get the average loss value over epcoh
                    _loss_pr_matrix = _loss_pr_matrix / e     # Get the average loss value over epoch
                    _loss_tr_matrix = np.concatenate((_loss_tr_matrix[:len(labels[0]), :], labels[0].reshape((-1, 1))), axis=1)
                    _loss_pr_matrix = np.concatenate((_loss_pr_matrix[:len(labels[1]), :], labels[1].reshape((-1, 1))), axis=1)
                    _loss_matrix = np.concatenate((_loss_tr_matrix, _loss_pr_matrix), axis=0)

            tra_accuracy = acc_eval(mlp, train_loader, train_set_len)
            val_accuracy = acc_eval(mlp, valid_loader, valid_set_len)
            tra_acc.append(tra_accuracy.cpu())
            val_acc.append(val_accuracy.cpu())

            if verbose:
                print(
                    f'[{e}] loss: {running_loss / i:.3f} \t training_accuracy: {tra_accuracy:.3f} \t validation_accuracy: {val_accuracy:.3f}')

            # Save parameter weights if they achieve optimal validation accuracy
            if val_accuracy >= max(val_acc):
                torch.save(mlp.state_dict(), file_name)

    # Regular training pipeline without data from pre_loader for generative replay
    else:
        for e in range(1, epoch + 1):
            # Set the model to train state
            mlp.train()
            # Initialize the running loss of the current epoch to 0.0
            running_loss = 0.0

            for i, (x, y) in enumerate(train_loader, 1):
                # Zero out the gradients
                """TO DO"""

                # Cast the features to `device`
                """TO DO"""

                # Cast the labels to `device`
                """TO DO"""


                # Calculate the loss
                """TO DO"""
                loss =
                # Back propagate the gradient
                """TO DO"""


                # Update the model
                """TO DO"""

                # Accumulate the running loss
                running_loss += loss.item()

                # Save training loss for data selection later
                if save_loss:
                    _lossF = CrossEntropyLoss(reduction='none')
                    _loss = _lossF(mlp(x), y).cpu().detach().numpy().reshape((-1, 1))
                    if i == 1:
                        tmp_loss_matrix = np.copy(_loss)
                    else:
                        tmp_loss_matrix = np.concatenate((tmp_loss_matrix, _loss), axis=0)

            # Save training loss for data selection later
            if save_loss:
                if e == 1:
                    _loss_matrix = np.copy(tmp_loss_matrix)
                else:
                    _loss_matrix += tmp_loss_matrix     # Accumulate loss
                if e == epoch:
                    _loss_matrix = _loss_matrix / e     # Get the average loss value over epoch
                    _loss_matrix = np.concatenate((_loss_matrix, labels.reshape((-1, 1))), axis=1)

            tra_accuracy = acc_eval(mlp, train_loader, train_set_len)
            val_accuracy = acc_eval(mlp, valid_loader, valid_set_len)
            tra_acc.append(tra_accuracy.cpu())
            val_acc.append(val_accuracy.cpu())

            if verbose:
                print(
                    f'[{e}] loss: {running_loss / i:.3f} \t training_accuracy: {tra_accuracy:.3f} \t validation_accuracy: {val_accuracy:.3f}')

            # Save parameter weights if they achieve optimal validation accuracy
            if val_accuracy >= max(val_acc):
                torch.save(mlp.state_dict(), file_name)
    if save_loss:
        return tra_acc, val_acc, _loss_matrix
    else:
        return tra_acc, val_acc

##Define some helper functions: (1 pt)

We define some helper functions here for plotting accuracy curves and printing  metrics.

In [ ]:
# Plot accuracy curves for training and validation accuracy
def plot_acc(t_acc, v_acc, file_name):
    e = range(1, len(t_acc) + 1)
    plt.figure()
    plt.plot(e, t_acc, 'b', label='Training Accuracy')
    plt.plot(e, v_acc, 'r', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.savefig(file_name)
    plt.show()

# Print accuracy metrics
def print_acc(mlp, train_loader, train_set_len, valid_loader, valid_set_len, mission1_test_loader, mission1_test_set_len, mission2_test_loader, mission2_test_set_len):
    '''
        Print accuracy metrics.

        Args:
            mlp:                     The MLP model
            train_loader:            DataLoader for training data
            train_set_len:           Number of data in the training set
            valid_loader:            DataLoader for validation data
            valid_set_len:           Number of data in the validation set
            mission1_test_loader:    DataLoader for Mission-1 test data
            mission1_test_set_len:   Number of data in the Mission-1 test set
            mission2_test_loader:    DataLoader for Mission-2 test data
            mission2_test_set_len:   Number of data in the Mission-2 test set
        Returns:
            accuracy1:               Accuracy on the Mission-1 test set (for BWT calculation)
    '''
    # Use acc_eval() to print training accuracy
    """TO DO"""
    accuracy =
    print(f'Training accuracy:        {accuracy:.3f}')

    # Use acc_eval() to print validation accuracy
    """TO DO"""
    accuracy =
    print(f'Validation accuracy:      {accuracy:.3f}')

    # Use acc_eval() to print Mission-1 test accuracy
    """TO DO"""
    accuracy1 =
    print(f'Mission-1 test accuracy:   {accuracy1:.3f}')

    # Use acc_eval() to print Mission-2 test accuracy
    """TO DO"""
    accuracy2 =
    print(f'Mission-2 test accuracy:   {accuracy2:.3f}')

    # Calculate the average test accuracy for both Mission-1 and Mission-2
    """TO DO"""
    avg_accuracy =
    print(f'Average test accuracy:    {avg_accuracy:.3f}')

    return accuracy1.cpu().item()

# Print average F1-score
def avg_F_metrics(mlp, experiment):
    '''
        Print F1-score metrics.

        Args:
            mlp:            The MLP model
            experiment:     Specify the experiment conducted:
                            ['DIL': for domain-incremental learning,
                             'CIL': for class-incremental learning]
        Returns:
            None
    '''
    mlp.eval()
    with torch.no_grad():
        x1 = torch.as_tensor(x_test_1).to(device)
        x2 = torch.as_tensor(x_test_2).to(device)
        y_pred_1 = torch.argmax(mlp(x1), 1).cpu().view(-1, 1)
        y_pred_2 = torch.argmax(mlp(x2), 1).cpu().view(-1, 1)
        y_real_1 = torch.as_tensor(y_test_1).view(-1, 1)
        y_real_2 = torch.as_tensor(y_test_2).view(-1, 1)
    print(f"Classification Report for Mission-1: \n {classification_report(y_real_1, y_pred_1, labels=[0, 1, 2], target_names=['class 0', 'class 1', 'class 2'], zero_division=0.0)}")
    print(f"Classification Report for Mission-2: \n {classification_report(y_real_2, y_pred_2, labels=[0, 1, 2], target_names=['class 0', 'class 1', 'class 2'], zero_division=0.0)}")
    mission1_report = classification_report(y_real_1, y_pred_1, labels=[0, 1, 2], target_names=['class 0', 'class 1', 'class 2'], zero_division=0.0, output_dict=True)
    mission2_report = classification_report(y_real_2, y_pred_2, labels=[0, 1, 2], target_names=['class 0', 'class 1', 'class 2'], zero_division=0.0, output_dict=True)
    if experiment == 'CIL':
        print(f"Average F1-Score:   {(mission1_report['weighted avg']['f1-score'] + mission2_report['weighted avg']['f1-score']) / 2:.3f}")
    else:
        print(f"Average F1-Score:   {(mission1_report['macro avg']['f1-score'] + mission2_report['macro avg']['f1-score']) / 2:.3f}")


## Initialize model parameters:  (0.5 pt)

We initialize the hyperparameters for our MLP model here.

In [ ]:
# Hyperparameters for our MLP model
"""TO DO"""
in_num =           # The number of input features
hidden_num1 = 256     # Number of neurons in hidden layer 1
hidden_num2 = 128     # Number of neurons in hidden layer 2
hidden_num3 = 128     # Number of neurons in hidden layer 3
out_num =            # Number of output classes

## Build the baseline model: (1 pt)

Here, we train our MLP model with data from Mission-1 (Domain-1) only. This is our baseline model that has only seen Mission-1 (Domain-1).

Use the `train_model()` function defined above to train our MLP model.

In addition, we set `save_loss = True` and assign `labels = y_train_1` to obtain a `loss_matrix` that records the average training loss values of all training data instances for our data preservation continual learning method later.

In [ ]:
# Random states
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
if device == 'cuda':
  torch.cuda.manual_seed(random_state)
  torch.cuda.manual_seed_all(random_state)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# Instansiate the model and cast it to `device`
"""TO DO"""
mlp =

# Use train_model() to train the mlp model with data from Mission-1 (Domain-1)
# Store the optimal model weights to the file 'domain_1.pt'
# Specify save_loss=True and labels=y_train_1 to record the average training
# loss values of the training data in loss_matrix
"""TO DO"""
tra_acc, val_acc, loss_matrix =

## Evaluate performance: (1 pt)

Now, we evaluate the MLP model's performance on both domains. You should see that our baseline model does a good job on the test set from Mission-1 (Domain-1), but it performs poorly on the test set from Mission-2 (Domain-2).

In [ ]:
# Load the optimal weights that gives the highest validation accuracy during training
"""TO DO"""


# Make a plot for the training and validation accuracy
plot_acc(tra_acc, val_acc, 'domain_1_acc')

print()
print("#### Results from Training on Domain 1 Only ####")
# Use print_acc() to print the accuracy metrics on the training, validation, and test sets from Mission-1 (Domain-1), and the test set from Mission-2 (Domain-2)
"""TO DO"""
baseline_acc_domain1 =

# Use avg_F_metrics() to print the F1-score metric
"""TO DO"""


## Naive fine-tuning: (1 pt)

Now, we naively fine-tune our baseline model with data from Mission-2 (Domain-2). You should observe that the model performs very well on the test set from Mission-2 (Domain-2) now, but performs very badly on the test set from Mission-1 (Domain-1). The performance on data from Mission-1 (Domain-1) deteriorate greatly. This is ***catastrophic forgetting***.

In addition, we report an additional metric, called **backward transfer**, to evaluate the performance of the continual learning algorithm. When learning a new mission, backward transfer measures how much the continual learning algorithm impacts the performance of the model on previous missions. See Section 3.2 in [DOCTOR](https://dl.acm.org/doi/full/10.1145/3679050) for details about backward transfer.

In [ ]:
# Random states
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
if device == 'cuda':
  torch.cuda.manual_seed(random_state)
  torch.cuda.manual_seed_all(random_state)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# load baseline model
"""TO DO"""

# Train the model with data from Mission-2 (Domain-2)
# Make sure to give a different file_name to store the optimal weights for this naively fine-tuned model
# We will use our baseline model's optimal weights, domain_1.pt, in the latter experiments
# Store the optimal model weights to the file 'domain_2.pt'
# Keep default values for verbose=True, save_loss=False, labels=None, pre_loader=None
"""TO DO"""
tra_acc, val_acc =

# Load the optimal weights that gives the highest validation accuracy during naive fine-tuning
"""TO DO"""

print()
print("#### Results from Naively Fine-tune on Domain 2 Only ####")
# Use print_acc() to print the accuracy metrics on the training, validation, and test sets from Mission-2 (Domain-2), and the test set from Mission-1 (Domain-1)
naive_acc_domain1 =
print()
# Use avg_F_metrics() to print the F1-score metric
"""TO DO"""

print()
# Calculate and print the backward transfer metric
"""TO DO"""
naive_bwt =
print(f"Naive Fine-Tuning Backward Transfer:    {naive_bwt:.3f}")

## Domain-incremental learning using the data preservation (DP) method:

Here, we will use the DP method to preserve training data from Mission-1 (Domain-1) for replay. When the model learns about a new domain (Domain-2), we will replay the preserved data from Mission-1 (Domain-1) to help our model retain the learned knowledge from Mission-1 (Domain-1).

## Define functions for the DP method: (2 pts)

Here, we define the functions for the DP algorithm. It preserves training data instances whose average training loss values are above a user-defined percentile in a stratified fashion.

Note:

Please read train_model() function to see what is returned in loss\_matix.

loss\_matix has 2 columns (column refers to dim=1): The first column is the average training loss, the last column is the y labels.

You will pass loss\_matix into preserve_data()

Hint: You may need to use Boolean indexing in Numpy.

In [ ]:
def get_boundary(loss):
    '''
    Find the threshold value based on the given loss matrix.

    Args:
        loss:         Loss matrix
    Returns:
        threshold:    The threshold value of a user-defined percentile
    '''
    # Find the value of the 70-th percentile average training loss values
    threshold = np.percentile(loss[:, -2], 70)
    return threshold


def preserve_data(x_data, y_data, _loss_matrix):
    '''
    Preserve data that have higher average training loss than the threshold value for replay.

    Args:
        x_data:         x data
        y_data:         y labels
        _loss_matrix:   The average training loss matrix
                  It has 2 columns (column refers to dim=1): The first column is the average training loss, the last column is the y labels.
                  Please check the returned value of train_model()
    Returns:
        x_preserve:     The preserved x data
        y_preserve:     The preserved y labels
    '''
    index_0, index_1, index_2 = [], [], []

    # For class 0:
    if sum(_loss_matrix[:, -1] == 0) > 0:
        # Find the threshold value based on a user-defined percentile for class 0 with the get_boundary() function
        # Pass the rows that belong to class 0 to get_boundary()
        # hint: You can achieve this using Boolean indexing in Numpy. Google for more information
        threshold_0 = get_boundary("""TO DO""")
        # For data instances belonging to class 0,
        # get a list of indices of data instances whose average training loss values are above the threshold value
        # Find the row indices that satisfy two conditions:
        # 1. belong to class 0
        # 2. average training loss values are greater than or equal to the threshold value
        index_0 = list(np.where("""TO DO""")[0])

    # For class 1:
    if sum(_loss_matrix[:, -1] == 1) > 0:
        # Find the threshold value based on a user-defined percentile for class 1 with the get_boundary() function
        # Pass the rows that belong to class 1 to get_boundary()
        threshold_1 = get_boundary("""TO DO""")
        # For data instances belonging to class 1,
        # get a list of indices of data instances whose average training loss values are above the threshold value
        # Find the row indices that satisfy two conditions:
        # 1. belong to class 1
        # 2. average training loss values are greater than or equal to the threshold value
        index_1 = list(np.where("""TO DO""")[0])

    # For class 2:
    if sum(_loss_matrix[:, -1] == 2) > 0:
        # Find the threshold value based on a user-defined percentile for class 2 with the get_boundary() function
        # Pass the rows that belong to class 2 to get_boundary()
        threshold_2 = get_boundary("""TO DO""")
        # For data instances belonging to class 2,
        # get a list of indices of data instances whose average training loss values are above the threshold value
        # Find the row indices that satisfy two conditions:
        # 1. belong to class 2
        # 2. average training loss values are greater than or equal to the threshold value
        index_2 = list(np.where("""TO DO""")[0])

    x_preserve = x_data[index_0 + index_1 + index_2, :]
    y_preserve = y_data[index_0 + index_1 + index_2]
    return x_preserve, y_preserve

## Preserve data: (0.5 pt)

Now, we will use the `preserve_data()` function defined above to preserve data from our Mission-1 (Domain-1) training set `(x_train_1, y_train_1)` based on the `loss_matrix` obtained from training the baseline model.

In [ ]:
# Use preserve_data() to preserve data for replay
x_train_1_preserve, y_train_1_preserve = preserve_data(x_train_1, y_train_1, loss_matrix)

# Create CustomDataset for the preserved training data
"""TO DO"""
domain1_preserve =
# Create DataLoader for domain1_preserve
# Halve the batch_size for this DataLoader so that the model will be trained with
# 50% of the data from domain1_preserve and 50% of the data from train_2 in each batch
# Set shuffle=True
"""TO DO"""
domain1_preserve_loader =

## Continual learning with the DP method: (1.5 pts)

Our baseline model is the model that has only learned from Mission-1 (Domain-1). Now, we will apply the DP continual learning algorithm to it to learn data from Mission-2 (Domain-2). You should observe that now our model performs well on both test sets from the two domains.

In [ ]:
# Random states
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
if device == 'cuda':
  torch.cuda.manual_seed(random_state)
  torch.cuda.manual_seed_all(random_state)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# Load the weights of the baseline model to ensure that we apply continual learning
# to our model that has only seen data from Mission-1 (Domain-1) (domain_1.pt)
"""TO DO"""


# Train the MLP model with Mission-2 (Domain-2) data and the preserved data
tra_acc, val_acc = train_model(mlp, epoch, train_loader_2, valid_loader_2, len(train_2), len(valid_2),
                               file_name=f'dp.pt', pre_loader=domain1_preserve_loader)

# Load the optimal weights that gives the highest validation accuracy during continual learning
"""TO DO"""


print()
print("######### Results from Continual Learning Using the DP Method #########")
# Use print_acc() to print the accuracy metrics on the training, validation, and test sets from Mission-2 (Domain-2), and the test set from Mission-1 (Domain-1)
"""TO DO"""
dp_acc_domain1 =

print()
# Use avg_F_metrics() to print the F1-score metric
"""TO DO"""


print()
# Calculate and print the backward transfer metric
"""TO DO"""
dp_bwt =
print(f"DP Backward Transfer:    {dp_bwt:.3f}")

# Part 3: Class-Incremental Learning Experiment (5.5 pts)

##Load the experimental data and create the datasets and dataloaders:
The dataset has been processed into the training, validation, and test sets for the two missions for you. You just need to load the corresponding experimental data.

First, we use Numpy to load the experimental data. Then, we create the training, validation, and test sets using the `CustomDataset` class. Finally, we instantiate dataloaders with PyTorch's [DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

In [ ]:
# Unzip the dataset
!unzip "/content/drive/Shared drives/ECE477 datasets/Assignment6/diabdeep_CIL_data.zip" -d diabdeep_CIL_data

# Check out the experimental data
print('\n', os.listdir('diabdeep_CIL_data/'), '\n')

# Load the data features x from Mission-1 as type "float32"
"""TO DO"""
x_train_1 =
x_valid_1 =
x_test_1 =

# Load the data features x from Mission-2 as type "float32"
"""TO DO"""
x_train_2 =
x_valid_2 =
x_test_2 =

# Load the labels y from Mission-1 as type "int64"
"""TO DO"""
y_train_1 =
y_valid_1 =
y_test_1 =

# Load the labels y from Mission-2 as type "int64"
"""TO DO"""
y_train_2 =
y_valid_2 =
y_test_2 =

# Inspect the labels for both missions
print(f"Mission-1 labels: {np.unique(y_test_1)}")
print(f"Mission-2 labels: {np.unique(y_test_2)}")

# Instantiate the datasets with CustomDataset
"""TO DO"""
train_1 =
valid_1 =
test_1 =
train_2 =
valid_2 =
test_2 =

# Set the batch size to 128
batch_size = 128

# Instantiate the dataloaders
# Note that we shuffle all of our datasets except for train_loader_1
# since we need to record each training data instance's training loss
# over all epochs for data selection (details later)
# Note that we halve the batch size for train_loader_2
# since we will train the model with a balanced amount of data from both domains
# in each batch for continual learning
"""TO DO"""
train_loader_1 =
valid_loader_1 =
test_loader_1 =
train_loader_2 =
valid_loader_2 =
test_loader_2 =

## Build the baseline model: (1 pt)

Here, we train our MLP model with data from Mission-1 only. This is our baseline model that has only seen Mission-1. There are only two classes in Mission-1 (healthy and Type-I diabetic), so we will set the `out_num = 2` for our baseline model.

Use the `train_model()` function defined above to train our MLP model.

This time, we leave `save_loss = False` since we will use the synthetic data generation (SDG) module to perform generative replay continual learning. Therefore, we won't need a `loss_matrix`.

In [ ]:
# Random states
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
if device == 'cuda':
  torch.cuda.manual_seed(random_state)
  torch.cuda.manual_seed_all(random_state)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# Hyperparameters for our MLP model
"""TO DO"""
in_num =          # The number of input features
hidden_num1 = 256     # Number of neurons in hidden layer 1
hidden_num2 = 128     # Number of neurons in hidden layer 2
hidden_num3 = 128     # Number of neurons in hidden layer 3
out_num =            # Number of output classes

# Instansiate the model and cast it to `device`
"""TO DO"""
mlp =

# Use train_model() to train the mlp model with data from Mission-1
# Store the optimal model weights to the file 'mission_1.pt'
# Keep default values for verbose=True, save_loss=False, labels=None, pre_loader=None
"""TO DO"""
tra_acc, val_acc =

## Evaluate performance: (1 pt)

Now, we evaluate the MLP model's performance on both missions. You should see that our baseline model does a good job on the test set from Mission-1, but it cannot perform on the test set from Mission-2.

In [ ]:
# Load the optimal weights that gives the highest validation accuracy during baseline training
"""TO DO"""


# Make a plot for the training and validation accuracy
plot_acc(tra_acc, val_acc, 'mission_1_acc')

print()
print("#### Results from Training on Mission 1 Only ####")
# Use print_acc() to print the accuracy metrics on the training, validation, and test sets from Mission-1, and the test set from Mission-2
"""TO DO"""
baseline_acc_mission1 =

# Use avg_F_metrics() to print the F1-score metric
"""TO DO"""


## Class-incremental learning using the SDG module with Gaussian mixture model estimation (GMME):

Here, we will use the SDG module to generate synthetic data representing Mission-1 for replay. We will use the GMME method to model the probability distribution of Mission-1 and sample synthetic data from the learned distribution. When the model learns about a new mission (Mission-2), we will replay the synthetic data to help our model retain the learned knowledge from Mission-1.

## GMME Synthetic data generation function: (1 pt)

We use [Gaussian mixture model](https://scikit-learn.org/1.5/modules/mixture.html) to build our synthetic data generation function. This section, we'll build our synthetic data generation function with `sklearn`'s [GaussianMixture](https://scikit-learn.org/1.5/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture).

In [ ]:
def GMME_sample_generation (X_train, X_validation, n_components=None):
    '''
    Use GMM to generate synthetic data.

    Args:
        X_train:        Training data
        X_validation:   Validation data
        n_components:   Number of Gaussian mixture components
    Returns:
        X_syn:          Generated synthetic data
    '''
    if not n_components:
        # Set a range of various n_components
        n_components = np.arange(5, 16, 1)
        score_array = np.zeros((len(n_components)))

        print("fitting GMM models with various components")
        for i, n in enumerate(n_components):
            # Fit a GMM to the training data with n mixture components
            # Use covariance_type='full' and random_state=random_state
            """TO DO"""
            gmm =
            # Compute the per-sample average log-likelihood of the validation data to find the optimal n_components, check documentation for available functions
            """TO DO"""
            score_array[i] =

        # Set the n_components to the one leading to max log-likelihood
        """TO DO"""
        n_components =

    print(f"Number of components: {n_components}")
    # Fit the GMM to the training data with n_components mixture components, 'full' covariance_type, and random_state=random_state
    """TO DO"""
    gmm =

    # Sample synthetic data from the GMM
    # The amount of synthetic data should be the same as the training data
    """TO DO"""
    X_syn, y_syn =

    return X_syn.astype(np.float32)

## Generate synthetic data: (1 pt)

Now, we will use the GMME function to generate the synthetic data we need to perform generative replay. The synthetic data represents the probability distribution of Mission-1.

In [ ]:
# Load the weights of the baseline model (mission_1.pt)
"""TO DO"""


# Use x_train_1 and x_valid_1 to generate synthetic training data gmm_x_syn
"""TO DO"""
gmm_x_syn =

# Use the baseline model to give gmm_x_syn their pseudo-labels gmm_y_syn
mlp.eval()
with torch.no_grad():
    x = torch.from_numpy(gmm_x_syn).to(device)
    # Get pseudo-labels gmm_y_syn as torch long integers
    # Transform the model's output prediction to 0, 1, or 2
    """TO DO"""
    gmm_y_syn =

# Create CustomDataset for the synthetic training data
"""TO DO"""
train_syn =

# Create DataLoader for train_syn
# Halve the batch_size for this DataLoader so that the model will be trained with
# 50% of the data from train_syn and 50% of the data from train_2 in each batch
# Set shuffle=True
"""TO DO"""
train_loader_syn =

## Create a new MLP model:

Next, we will apply generative replay continual learning to our baseline model. But first of all, we need to add one more neuron to the output layer so the model can learn a new class. Then, we will train our baseline model with data from Mission-2 while replaying the synthetic data we just generated.

Here, we will use the same MLP structure, but we will add one more neuron to the output layer.



In [ ]:
class NewMLP(Module):
    def __init__(self, in_num, out_num, hidden_num1, hidden_num2, hidden_num3):
        super(NewMLP, self).__init__()

        # Add the first hidden layer and ReLU activation
        """TO DO"""
        self.hidden1 =
        self.relu1 =

        # Add the second hidden layer and ReLU activation
        """TO DO"""
        self.hidden2 =
        self.relu2 =

        # Add the third hidden layer and ReLU activation
        """TO DO"""
        self.hidden3 =
        self.relu3 =

        # Add the new output layer
        """TO DO"""
        self.out_new =

    def forward(self, x):
        # Pass x forward the first hidden layer and ReLU activation
        """TO DO"""

        # Pass x forward the second hidden layer and ReLU activation
        """TO DO"""

        # Pass x forward the third hidden layer and ReLU activation
        """TO DO"""

        # Pass x forward the new output layer
        """TO DO"""

        return x

## Continual learning with the GMME SDG: (1.5 pts)

Our baseline model is the model that has only learned from Mission-1. Now, we will apply the GMME SDG continual learning algorithm to it to learn data from Mission-2. You should observe that now our model performs well on both test sets from the two missions.

In [ ]:
# Random states:
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
if device == 'cuda':
  torch.cuda.manual_seed(random_state)
  torch.cuda.manual_seed_all(random_state)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# Add one more neuron to the output layer
"""TO DO"""
out_num =            # Number of output classes

# Re-instansiate the model as the NewMLP class and cast it to `device`
"""TO DO"""
mlp_new =

# Load the weights of the baseline model to mlp_new (mission_1.pt)
# Note that mlp_new includes one additional output neuron. When loading the weights, ensure that you ignore the additional neuron in mlp_new to avoid errors
# Please check the official PyTorch documentation on load_state_dict here:
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.load_state_dict
"""TO DO"""


# Note that we will plug train_loader_syn in pre_loader and leave save_loss=False
tra_acc, val_acc = train_model(mlp_new, epoch, train_loader_2, valid_loader_2, len(train_2), len(valid_2), file_name=f'gmm.pt', pre_loader=train_loader_syn)

# Load the optimal weights that gives the highest validation accuracy during continual learning
"""TO DO"""


print()
print("#### Results from Continual Learning Using the SDG Module with GMME ####")
# Use print_acc() to print the accuracy metrics on the training, validation, and test sets from Mission-2, and the test set from Mission-1
"""TO DO"""
gmme_acc_mission1 =

# Use avg_F_metrics() to print the F1-score metric
"""TO DO"""


print()
# Calculate and print the backward transfer metric
"""TO DO"""
gmme_bwt =

print(f"GMME Backward Transfer:    {gmme_bwt:.3f}")